In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

bronze_table = "workspace.air_quality.bronze_air_quality"
silver_table = "workspace.air_quality.silver_air_quality"

silver_df = spark.table(bronze_table)
print(f"Loaded {silver_df.count():,} rows")
silver_df.printSchema()

Loaded 14,296,116 rows
root
 |-- from_date: string (nullable = true)
 |-- to_date: string (nullable = true)
 |-- pm25_ugm3: string (nullable = true)
 |-- pm10_ugm3: string (nullable = true)
 |-- no_ugm3: string (nullable = true)
 |-- no2_ugm3: string (nullable = true)
 |-- nox_ppb: string (nullable = true)
 |-- nh3_ugm3: string (nullable = true)
 |-- so2_ugm3: string (nullable = true)
 |-- co_mgm3: string (nullable = true)
 |-- ozone_ugm3: string (nullable = true)
 |-- benzene_ugm3: string (nullable = true)
 |-- toluene_ugm3: string (nullable = true)
 |-- ethylbenzene_ugm3: string (nullable = true)
 |-- mp_xylene_ugm3: string (nullable = true)
 |-- o_xylene_ugm3: string (nullable = true)
 |-- temp_c: string (nullable = true)
 |-- relative_humidity_pct: string (nullable = true)
 |-- wind_speed_ms: string (nullable = true)
 |-- wind_direction_deg: string (nullable = true)
 |-- solar_radiation_wmt2: string (nullable = true)
 |-- barometric_pressure_mmhg: string (nullable = true)
 |-- vert

In [0]:
from pyspark.sql import functions as F

silver_clean = (silver_df
    .withColumn("from_timestamp", F.try_to_timestamp("from_date"))
    .withColumn("to_timestamp", F.try_to_timestamp("to_date"))
    .withColumn("date", F.try_to_date("from_date"))
    
    .withColumn("pm25_ugm3", F.col("pm25_ugm3").cast("double"))
    .withColumn("pm10_ugm3", F.col("pm10_ugm3").cast("double"))
    .withColumn("no2_ugm3", F.col("no2_ugm3").cast("double"))
    .withColumn("no_ugm3", F.col("no_ugm3").cast("double"))
    .withColumn("so2_ugm3", F.col("so2_ugm3").cast("double"))
    .withColumn("co_mgm3", F.col("co_mgm3").cast("double"))
    .withColumn("ozone_ugm3", F.col("ozone_ugm3").cast("double"))
    
    .withColumn("temp_c", F.col("temp_c").cast("double"))
    .withColumn("relative_humidity_pct", F.col("relative_humidity_pct").cast("double"))
    .withColumn("wind_speed_ms", F.col("wind_speed_ms").cast("double"))
    .withColumn("wind_direction_deg", F.col("wind_direction_deg").cast("double"))
    .withColumn("air_temp_c", F.col("air_temp_c").cast("double"))
    
    .filter(F.col("date").isNotNull())
    .dropDuplicates(["city_code", "state_code", "date"])
    
    .select("city_code", "state_code", "date", "from_timestamp", "to_timestamp",
            "pm25_ugm3", "pm10_ugm3", "no2_ugm3", "so2_ugm3", "co_mgm3", 
            "ozone_ugm3", "temp_c", "relative_humidity_pct", "wind_speed_ms",
            "wind_direction_deg", "air_temp_c", "source_file", "ingest_ts")
)

print(f"Silver: {silver_clean.count():,} valid rows")
display(silver_clean.limit(10))
silver_clean.printSchema()

Silver: 595,882 valid rows


city_code,state_code,date,from_timestamp,to_timestamp,pm25_ugm3,pm10_ugm3,no2_ugm3,so2_ugm3,co_mgm3,ozone_ugm3,temp_c,relative_humidity_pct,wind_speed_ms,wind_direction_deg,air_temp_c,source_file,ingest_ts
HR001,HR,2013-03-12,2013-03-12T00:00:00.000Z,2013-03-12T01:00:00.000Z,null,23.2,70.82,81.84,0.0,0.0,745.65,0.14,0.0,null,null,dbfs:/Volumes/workspace/air_quality/data/HR001.csv,2026-02-01T09:53:10.297Z
HR001,HR,2013-09-25,2013-09-25T00:00:00.000Z,2013-09-25T01:00:00.000Z,null,10.21,12.53,1.01,0.0,0.0,756.1,0.1,0.0,0.0,null,dbfs:/Volumes/workspace/air_quality/data/HR001.csv,2026-02-01T09:53:10.297Z
HR001,HR,2014-07-12,2014-07-12T00:00:00.000Z,2014-07-12T01:00:00.000Z,null,6.49,13.78,4.75,0.0,0.0,724.52,-0.13,0.0,null,null,dbfs:/Volumes/workspace/air_quality/data/HR001.csv,2026-02-01T09:53:10.297Z
HR001,HR,2016-02-19,2016-02-19T00:00:00.000Z,2016-02-19T01:00:00.000Z,213.0,26.51,55.36,5.8,1.27,3.0,737.6,-0.14,0.0,1.11,null,dbfs:/Volumes/workspace/air_quality/data/HR001.csv,2026-02-01T09:53:10.297Z
HR001,HR,2016-04-08,2016-04-08T00:00:00.000Z,2016-04-08T01:00:00.000Z,75.0,13.3,15.83,17.84,0.4,1.02,732.0,-0.08,0.0,0.32,null,dbfs:/Volumes/workspace/air_quality/data/HR001.csv,2026-02-01T09:53:10.297Z
HR002,HR,2022-09-15,2022-09-15T00:00:00.000Z,2022-09-15T01:00:00.000Z,null,null,null,null,null,null,null,null,null,null,null,dbfs:/Volumes/workspace/air_quality/data/HR002.csv,2026-02-01T09:53:10.297Z
HR003,HR,2017-05-29,2017-05-29T00:00:00.000Z,2017-05-29T01:00:00.000Z,null,null,null,null,null,null,null,null,null,null,null,dbfs:/Volumes/workspace/air_quality/data/HR003.csv,2026-02-01T09:53:10.297Z
HR003,HR,2019-06-25,2019-06-25T00:00:00.000Z,2019-06-25T01:00:00.000Z,20.84,null,10.93,0.54,12.05,0.69,63.75,1.46,188.94,52.88,null,dbfs:/Volumes/workspace/air_quality/data/HR003.csv,2026-02-01T09:53:10.297Z
HR004,HR,2016-04-10,2016-04-10T00:00:00.000Z,2016-04-10T01:00:00.000Z,null,null,null,null,null,null,null,null,null,null,null,dbfs:/Volumes/workspace/air_quality/data/HR004.csv,2026-02-01T09:53:10.297Z
HR004,HR,2021-04-04,2021-04-04T00:00:00.000Z,2021-04-04T01:00:00.000Z,45.04,null,76.25,0.89,63.95,4.12,29.54,1.31,161.15,43.6,null,dbfs:/Volumes/workspace/air_quality/data/HR004.csv,2026-02-01T09:53:10.297Z


root
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- date: date (nullable = true)
 |-- from_timestamp: timestamp (nullable = true)
 |-- to_timestamp: timestamp (nullable = true)
 |-- pm25_ugm3: double (nullable = true)
 |-- pm10_ugm3: double (nullable = true)
 |-- no2_ugm3: double (nullable = true)
 |-- so2_ugm3: double (nullable = true)
 |-- co_mgm3: double (nullable = true)
 |-- ozone_ugm3: double (nullable = true)
 |-- temp_c: double (nullable = true)
 |-- relative_humidity_pct: double (nullable = true)
 |-- wind_speed_ms: double (nullable = true)
 |-- wind_direction_deg: double (nullable = true)
 |-- air_temp_c: double (nullable = true)
 |-- source_file: string (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)



In [0]:
(silver_clean.write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable(silver_table)
)

print("Silver table created!")
spark.sql(f"SELECT city_code, state_code, COUNT(*) as records FROM {silver_table} GROUP BY city_code, state_code ORDER BY records DESC LIMIT 10").display()

Silver table created!


city_code,state_code,records
UP001,UP,4838
UP002,UP,4838
TN003,TN,4838
MH002,MH,4838
TN001,TN,4838
TN002,TN,4838
KA003,KA,4838
KA002,KA,4838
MH001,MH,4838
MH003,MH,4838
